# Notebook 03: Two-State Vector Formalism & Weak Values

**References**:
- Aharonov, Bergmann, Lebowitz (1964), Phys. Rev. B 134, 1410
- Aharonov, Albert, Vaidman (1988), PRL 60, 1351

## The Big Idea

Standard QM describes a system by one state vector $|\psi\rangle$ evolving **forward** in time. The TSVF describes it by **two** state vectors:

$$\langle\Phi| \quad \cdot \quad |\Psi\rangle$$

- $|\Psi\rangle$: evolves **forward** from preparation (past boundary)
- $\langle\Phi|$: evolves **backward** from post-selection (future boundary)

This leads to the **weak value**:

$$A_w = \frac{\langle\Phi|\hat{A}|\Psi\rangle}{\langle\Phi|\Psi\rangle}$$

Weak values can be **complex**, **anomalous** (outside the eigenvalue range), and are **experimentally measurable** via weak measurements.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

from src.core.operators import PAULI
from src.tsvf.two_state_vector import TwoStateVector
from src.tsvf.weak_values import WeakValueCalculator
from src.tsvf.abl_rule import ABLRule

## 1. Basic Weak Value Computation

For a spin-1/2 particle:
- Pre-select: $|\psi\rangle = |\uparrow_z\rangle = |0\rangle$
- Post-select: $|\phi\rangle = |\uparrow_x\rangle = |+\rangle = (|0\rangle + |1\rangle)/\sqrt{2}$
- Observable: $\hat{\sigma}_z$ (eigenvalues $\pm 1$)

The weak value is:
$$\sigma_{z,w} = \frac{\langle +|\sigma_z|0\rangle}{\langle +|0\rangle} = \frac{1/\sqrt{2}}{1/\sqrt{2}} = 1$$

This is within the eigenvalue range. Now let's find **anomalous** weak values.

In [ ]:
# Pre-select |0> (spin up along z)
pre = np.array([1, 0], dtype=complex)

# Post-select |+> (spin up along x)
post = np.array([1, 1], dtype=complex) / np.sqrt(2)

tsv = TwoStateVector(pre, post)
calc = WeakValueCalculator(tsv)

# Compute weak values of all Pauli operators
print("Pre-select: |0> (spin up z)")
print("Post-select: |+> (spin up x)")
print("=" * 50)

for name, op in [('sigma_x', PAULI['X']), ('sigma_y', PAULI['Y']), ('sigma_z', PAULI['Z'])]:
    wv = calc.compute(op)
    anomalous = calc.is_anomalous(op)
    print(f"  {name}_w = {wv.real:+.4f} {wv.imag:+.4f}i"
          f"  {'** ANOMALOUS **' if anomalous else '(normal)'}")
    print(f"    Eigenvalue range: [-1, +1], Re(A_w) = {wv.real:+.4f}")

## 2. Anomalous Weak Values: Spin 100

The famous AAV (1988) result: by choosing nearly-orthogonal pre/post states, the weak value can be **far** outside the eigenvalue range. A spin-1/2 particle can have $\sigma_{z,w} = 100$!

In [ ]:
# Sweep the angle between pre and post states
# As they approach orthogonality, the weak value diverges
angles = np.linspace(0.01, np.pi/2, 200)
wv_real = []
wv_imag = []

for theta in angles:
    pre = np.array([1, 0], dtype=complex)
    post = np.array([np.cos(theta), np.sin(theta)], dtype=complex)
    
    tsv = TwoStateVector(pre, post)
    calc = WeakValueCalculator(tsv)
    wv = calc.compute(PAULI['Z'])
    wv_real.append(wv.real)
    wv_imag.append(wv.imag)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

ax1.plot(np.degrees(angles), wv_real, 'b-', linewidth=2)
ax1.axhline(y=1, color='red', linestyle='--', label='Eigenvalue +1')
ax1.axhline(y=-1, color='red', linestyle='--', label='Eigenvalue -1')
ax1.fill_between(np.degrees(angles), -1, 1, alpha=0.1, color='red', label='Normal range')
ax1.set_xlabel('Post-selection angle (degrees)', fontsize=12)
ax1.set_ylabel('Re(sigma_z weak value)', fontsize=12)
ax1.set_title('Weak value vs post-selection angle', fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Complex plane
ax2.scatter(wv_real, wv_imag, c=np.degrees(angles), cmap='viridis', s=5)
ax2.axhline(y=0, color='gray', linewidth=0.5)
ax2.axvline(x=0, color='gray', linewidth=0.5)
circle = plt.Circle((0, 0), 1, fill=False, color='red', linestyle='--', label='Eigenvalue circle')
ax2.add_patch(circle)
ax2.set_xlabel('Re(A_w)', fontsize=12)
ax2.set_ylabel('Im(A_w)', fontsize=12)
ax2.set_title('Weak value in the complex plane', fontsize=13)
ax2.set_aspect('equal')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"At angle=1 degree: sigma_z_w = {wv_real[0]:.1f} (far outside [-1, +1]!)")

## 3. The Three-Box Paradox

The most striking demonstration of TSVF. A particle is in one of three boxes (A, B, C):

- Pre-select: $|\psi\rangle = (|A\rangle + |B\rangle + |C\rangle)/\sqrt{3}$
- Post-select: $|\phi\rangle = (|A\rangle + |B\rangle - |C\rangle)/\sqrt{3}$

Weak values of box projectors:
- $\Pi_{A,w} = 1$ → particle is "certainly in box A"
- $\Pi_{B,w} = 1$ → particle is "certainly in box B"
- $\Pi_{C,w} = -1$ → particle has **negative** probability in box C!

Yet $\Pi_{A,w} + \Pi_{B,w} + \Pi_{C,w} = 1$ (consistent).

In [ ]:
# Run the three-box paradox
# We need a dummy TSV to access the method
dummy_tsv = TwoStateVector(np.array([1, 0], dtype=complex), np.array([0, 1], dtype=complex))
dummy_calc = WeakValueCalculator(dummy_tsv)
result = dummy_calc.three_box_paradox()

print("THE THREE-BOX PARADOX")
print("=" * 60)
print(f"Pre-select:  |psi> = (|A> + |B> + |C>) / sqrt(3)")
print(f"Post-select: |phi> = (|A> + |B> - |C>) / sqrt(3)")
print()
print(f"  Pi_A weak value = {result['Pi_A_weak_value'].real:+.4f}")
print(f"  Pi_B weak value = {result['Pi_B_weak_value'].real:+.4f}")
print(f"  Pi_C weak value = {result['Pi_C_weak_value'].real:+.4f}")
print(f"\n  Sum = {result['sum'].real:+.4f} (must equal 1: {result['sum_equals_1']})")
print(f"\n  Pi_C is negative: {result['Pi_C_negative']}")
print(f"\n{result['explanation']}")

In [ ]:
# Visualize the three-box paradox
fig, ax = plt.subplots(figsize=(8, 5))

boxes = ['Box A', 'Box B', 'Box C', 'Sum']
values = [result['Pi_A_weak_value'].real, result['Pi_B_weak_value'].real,
          result['Pi_C_weak_value'].real, result['sum'].real]
colors = ['#2196F3', '#4CAF50', '#F44336', '#9C27B0']

bars = ax.bar(boxes, values, color=colors, edgecolor='black', linewidth=1.5, width=0.6)
ax.axhline(y=0, color='black', linewidth=0.8)
ax.axhline(y=1, color='gray', linestyle='--', alpha=0.5, label='Eigenvalue bounds [0, 1]')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

for bar, val in zip(bars, values):
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.05,
            f'{val:+.1f}', ha='center', va='bottom', fontsize=14, fontweight='bold')

ax.set_ylabel('Weak Value', fontsize=13)
ax.set_title('Three-Box Paradox: The particle is in A AND in B simultaneously,\n'
             'with NEGATIVE presence in C', fontsize=12)
ax.set_ylim(-1.5, 1.5)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 4. Weak Measurement Simulation

Weak values aren't just math — they are **experimentally measurable**. In a weak measurement:

1. Couple a pointer (Gaussian) weakly to the observable
2. Post-select the system
3. The pointer shifts by $g \cdot \text{Re}(A_w)$

When $A_w$ is anomalous, the pointer shifts **beyond** the normal range!

In [ ]:
# Weak measurement with anomalous weak value
pre = np.array([1, 0], dtype=complex)  # |0>
# Post-select at a small angle to get anomalous weak value
theta = 0.1  # small angle -> near-orthogonal -> anomalous
post = np.array([np.cos(np.pi/2 - theta), np.sin(np.pi/2 - theta)], dtype=complex)

tsv = TwoStateVector(pre, post)
calc = WeakValueCalculator(tsv)

# Simulate weak measurement of sigma_z
result = calc.weak_measurement_simulation(
    PAULI['Z'],
    coupling_strength=0.05,
    n_trials=100000,
    pointer_width=1.0
)

print(f"Weak value: {result.weak_value.real:+.4f} {result.weak_value.imag:+.4f}i")
print(f"Anomalous: {result.is_anomalous}")
print(f"Eigenvalue range: [{result.eigenvalues.min():.1f}, {result.eigenvalues.max():.1f}]")
print(f"\nPointer shift (theory): {result.pointer_shift_theory:.6f}")
print(f"Pointer shift (measured): {result.pointer_shift_measured:.6f}")
print(f"Post-selected trials: {result.n_trials}")

fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(result.pointer_readings, bins=80, density=True, alpha=0.7, 
        color='steelblue', edgecolor='white', label='Post-selected pointer')
ax.axvline(x=0, color='gray', linestyle='--', label='No coupling', alpha=0.5)
ax.axvline(x=result.pointer_shift_theory, color='red', linewidth=2,
           label=f'Theory: g*Re(A_w) = {result.pointer_shift_theory:.4f}')
ax.axvline(x=result.pointer_shift_measured, color='green', linewidth=2, linestyle='--',
           label=f'Measured mean = {result.pointer_shift_measured:.4f}')
ax.set_xlabel('Pointer position', fontsize=12)
ax.set_ylabel('Probability density', fontsize=12)
ax.set_title(f'Weak Measurement of sigma_z (A_w = {result.weak_value.real:.2f})', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. ABL Rule: Time-Symmetric Born Rule

The ABL (Aharonov-Bergmann-Lebowitz) rule gives the probability of intermediate measurement results for a **pre-and-post-selected** system:

$$P(a_j) = \frac{|\langle\phi|\Pi_j|\psi\rangle|^2}{\sum_k |\langle\phi|\Pi_k|\psi\rangle|^2}$$

Unlike the Born rule $P(a_j) = |\langle a_j|\psi\rangle|^2$, the ABL rule depends on **both** the past preparation and the future post-selection.

In [ ]:
# ABL vs Born rule comparison
pre = np.array([1, 0], dtype=complex)  # |0>
post = np.array([np.cos(0.3), np.sin(0.3)], dtype=complex)

tsv = TwoStateVector(pre, post)
abl = ABLRule(tsv)

comparison = abl.compare_with_born_rule(PAULI['Z'])

print("ABL Rule vs Born Rule for sigma_z")
print("=" * 50)
print(f"Pre-select:  |0>")
print(f"Post-select: cos(0.3)|0> + sin(0.3)|1>")
print()
print(f"{'Eigenvalue':>12} {'Born Rule':>12} {'ABL Rule':>12} {'Difference':>12}")
print("-" * 50)
for i, ev in enumerate(comparison.eigenvalues):
    print(f"{ev:>12.1f} {comparison.born_probabilities[i]:>12.4f} "
          f"{comparison.abl_probabilities[i]:>12.4f} "
          f"{comparison.abl_probabilities[i] - comparison.born_probabilities[i]:>+12.4f}")
print(f"\nMax difference: {comparison.max_difference:.4f}")
print(f"Time-symmetric: {comparison.time_symmetric}")
print("\n=> The ABL rule differs from Born when post-selection provides")
print("   additional information. The future CHANGES the prediction.")

In [ ]:
# Sweep the post-selection angle and show how ABL deviates from Born
post_angles = np.linspace(0.05, np.pi/2, 50)
abl_p0 = []
born_p0 = []

for theta in post_angles:
    post = np.array([np.cos(theta), np.sin(theta)], dtype=complex)
    tsv = TwoStateVector(pre, post)
    abl = ABLRule(tsv)
    comp = abl.compare_with_born_rule(PAULI['Z'])
    # Probability of finding eigenvalue +1 (spin up)
    abl_p0.append(comp.abl_probabilities[1])  # +1 eigenvalue
    born_p0.append(comp.born_probabilities[1])

fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(np.degrees(post_angles), born_p0, 'b--', linewidth=2, label='Born rule P(+1)')
ax.plot(np.degrees(post_angles), abl_p0, 'r-', linewidth=2, label='ABL rule P(+1)')
ax.fill_between(np.degrees(post_angles), born_p0, abl_p0, alpha=0.2, color='purple',
                label='Retrocausal contribution')
ax.set_xlabel('Post-selection angle (degrees)', fontsize=12)
ax.set_ylabel('P(sigma_z = +1)', fontsize=12)
ax.set_title('Born Rule vs ABL Rule: The Future Affects Predictions', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("The shaded region is the 'retrocausal contribution':")
print("knowledge of the FUTURE post-selection changes intermediate predictions.")

## 6. Time Symmetry Verification

The ABL rule is **time-symmetric**: swapping pre/post states and reversing the Hamiltonian gives the same probabilities. This is the mathematical foundation of retrocausality in TSVF.

In [ ]:
# Verify time symmetry with a non-trivial Hamiltonian
pre = np.array([1, 0], dtype=complex)
post = np.array([np.cos(0.5), np.sin(0.5)], dtype=complex)
H = 0.5 * PAULI['X']  # non-trivial evolution

tsv = TwoStateVector(pre, post, H)
abl = ABLRule(tsv)

demo = abl.time_symmetry_demonstration(PAULI['Z'], t=0.5)

print("Time Symmetry Demonstration")
print("=" * 50)
print(f"  Forward: |psi> -> H -> measurement -> |phi>")
print(f"  Reverse: |phi> -> (-H) -> measurement -> |psi>")
print(f"\n  Time-symmetric: {demo['time_symmetric']}")
print(f"  Max ABL-Born difference: {demo['max_abl_born_difference']:.4f}")
print(f"\n{demo['explanation']}")

## Key Takeaways

1. **Weak values** are experimentally measurable quantities that reveal what happens "between" measurements in quantum mechanics.

2. **Anomalous weak values** (outside the eigenvalue range) are the smoking gun of TSVF — they show that pre-and-post-selected systems have fundamentally different properties than standard quantum systems.

3. The **three-box paradox** shows a particle can be "certainly in A" AND "certainly in B" simultaneously, with negative presence in C.

4. The **ABL rule** is the time-symmetric generalization of the Born rule: the future post-selection genuinely changes intermediate predictions.

5. **Time symmetry** is exact: TSVF treats past and future on equal footing.

---

**Next**: [04_retrocausal_bell_test.ipynb](04_retrocausal_bell_test.ipynb) — Retrocausal models that violate Bell's inequality while remaining local.